In [27]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN
from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [7]:
data = yf.download("GOOG", period="20Y")
df = pd.DataFrame(data)

[*********************100%***********************]  1 of 1 completed


In [24]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2004-08-19,2.490664,2.591785,2.390042,2.499133,2.499133,897427216
2004-08-20,2.515820,2.716817,2.503118,2.697639,2.697639,458857488
2004-08-23,2.758411,2.826406,2.716070,2.724787,2.724787,366857939
2004-08-24,2.770615,2.779581,2.579581,2.611960,2.611960,306396159
2004-08-25,2.614201,2.689918,2.587302,2.640104,2.640104,184645512


In [10]:
# Preprocess data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [11]:
# Define training data length
training_data_len = math.ceil(len(data) * 0.8)

# Split data
train_data = scaled_data[:training_data_len]
test_data = scaled_data[training_data_len:]

# Function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data)-seq_length):
        X.append(data[i:(i+seq_length)])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)


## LSTM

In [12]:
# Create sequences for training
seq_length = 60
X_train, y_train = create_sequences(train_data, seq_length)

# Define LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=5, batch_size=32)

C:\Users\Karolina\anaconda3\envs\Magisterka\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - loss: 0.0039
Epoch 2/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 6.4192e-05
Epoch 3/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 5.6395e-05
Epoch 4/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 5.4794e-05
Epoch 5/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 4.3273e-05


In [21]:
# Evaluate model
X_test, y_test = create_sequences(test_data, seq_length)
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions).round(4)
print("Mean Squared Error:", mse)

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Mean Squared Error: 0.0011


In [22]:
rmse = root_mean_squared_error(y_test, predictions).round(4)
print("RMSE:", rmse)

RMSE: 0.0335


In [25]:
# predykcja na 1 dzień 
# Get the last sequence from the training data
last_sequence = train_data[-seq_length:]

# Reshape the last sequence to fit the model input shape
last_sequence = last_sequence.reshape(1, seq_length, 1)

# Make a prediction for the next day
next_day_prediction = model.predict(last_sequence)

# Inverse transform the prediction to get the actual price
next_day_prediction = scaler.inverse_transform(next_day_prediction)

print("Predicted price for the next day:", next_day_prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted price for the next day: [[70.11713]]


## RNN

In [28]:
# Define RNN model
model_rnn = Sequential()
model_rnn.add(SimpleRNN(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_rnn.add(SimpleRNN(units=50))
model_rnn.add(Dense(units=1))

# Compile RNN model
model_rnn.compile(optimizer='adam', loss='mean_squared_error')

# Train RNN model
model_rnn.fit(X_train, y_train, epochs=5, batch_size=32)

# Make prediction using RNN model
next_day_prediction_rnn = model_rnn.predict(last_sequence)

# Inverse transform the prediction to get the actual price
next_day_prediction_rnn = scaler.inverse_transform(next_day_prediction_rnn)

print("Predicted price for the next day using RNN:", next_day_prediction_rnn)

Epoch 1/5


C:\Users\Karolina\anaconda3\envs\Magisterka\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


123/123 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0214
Epoch 2/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 8.4653e-05
Epoch 3/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 6.9044e-05
Epoch 4/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 4.8480e-05
Epoch 5/5
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 5.9715e-05
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
Predicted price for the next day using RNN: [[68.444435]]
